<a href="https://colab.research.google.com/github/RaghavaDhanya/AnimeFaceAlign/blob/main/AnimeFaceAlign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RaghavaDhanya/AnimeFaceAlign/blob/main/AnimeFaceAlign.ipynb)

In [1]:
!git clone https://github.com/kanosawa/anime_face_landmark_detection.git

Cloning into 'anime_face_landmark_detection'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 35 (delta 13), reused 26 (delta 7), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [2]:
%cd anime_face_landmark_detection/

/content/anime_face_landmark_detection


In [3]:
!gdown https://drive.google.com/uc?id=1NckKw7elDjQTllRxttO87WY7cnQwdMqz

Downloading...
From: https://drive.google.com/uc?id=1NckKw7elDjQTllRxttO87WY7cnQwdMqz
To: /content/anime_face_landmark_detection/checkpoint_landmark_191116.pth.tar
16.7MB [00:00, 63.3MB/s]


In [4]:
!wget https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml

--2021-09-18 18:31:48--  https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246945 (241K) [text/plain]
Saving to: ‘lbpcascade_animeface.xml’

lbpcascade_animefac 100%[===================>] 241.16K  --.-KB/s    in 0.03s   

2021-09-18 18:31:48 (7.72 MB/s) - ‘lbpcascade_animeface.xml’ saved [246945/246945]



In [11]:
import numpy as np
import torch
from torchvision import transforms
import cv2
from PIL import Image, ImageDraw, ImageFont

from CFA import CFA
# import animeface


# param

def get_landmarks(img, face_detector, landmark_detector, img_width=128, num_landmark = 24):
# transform
    normalize   = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                    std=[0.5, 0.5, 0.5])
    train_transform = [transforms.ToTensor(), normalize]
    train_transform = transforms.Compose(train_transform)

    # input image & detect face
    # img = cv2.imread(input_img_name)

    faces = face_detector.detectMultiScale(cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB),scaleFactor =1.05)
    # draw = ImageDraw.Draw(img)

    for x_, y_, w_, h_ in faces:
        # adjust face size
        x = max(x_ - w_ / 8, 0)
        rx = min(x_ + w_ * 9 / 8, img.width)
        y = max(y_ - h_ / 4, 0)
        by = y_ + h_
        w = rx - x
        h = by - y

        # draw result of face detection
        # draw.rectangle((x, y, x + w, y + h), outline=(0, 0, 255), width=3)

        # transform image
        img_tmp = img.crop((x, y, x+w, y+h))
        img_tmp = img_tmp.resize((img_width, img_width), Image.BICUBIC)
        img_tmp = train_transform(img_tmp)
        img_tmp = img_tmp.unsqueeze(0).cuda()

        # estimate heatmap
        heatmaps = landmark_detector(img_tmp)
        heatmaps = heatmaps[-1].cpu().detach().numpy()[0]
        landmarks = []
        # calculate landmark position
        for i in range(num_landmark):
            heatmaps_tmp = cv2.resize(heatmaps[i], (img_width, img_width), interpolation=cv2.INTER_CUBIC)
            landmark = np.unravel_index(np.argmax(heatmaps_tmp), heatmaps_tmp.shape)
            landmark_y = landmark[0] * h / img_width
            landmark_x = landmark[1] * w / img_width
            landmarks.append([x+landmark_x, y+landmark_y])
        yield landmarks

# output image
# img.save('output.png')

In [16]:
import scipy.ndimage
from pathlib import Path

def align_image(img_path, output_size=1024, transform_size=4096, enable_padding=True, x_scale=1, y_scale=1, em_scale=0.1, alpha=False):
    checkpoint_name = 'checkpoint_landmark_191116.pth.tar'
    img_orig =Image.open(img_path) 
    num_landmark = 24

    # detector
    face_detector = cv2.CascadeClassifier('lbpcascade_animeface.xml')
    landmark_detector = CFA(output_channel_num=num_landmark + 1, checkpoint_name=checkpoint_name).cuda()
    for i, landmarks in enumerate(get_landmarks(img_orig, face_detector, landmark_detector, num_landmark=num_landmark)):
        img = img_orig.copy()

        lm = np.array(landmarks)
        face_outline = lm[[0, 1, 2]]
        left_eye_brow = lm[[3, 4, 5]]
        right_eye_brow = lm[[6, 7, 8]]
        nose = lm[[9]]
        lm_eye_left = lm[[10, 11, 12, 13, 14]]
        lm_eye_right = lm[[15, 16, 17, 18, 19]]
        mouth = lm[[20, 21, 22, 23]]

        mouth_left_point = lm[20]
        mount_right_point = lm[22]


        eye_left     = np.mean(lm_eye_left, axis=0)
        eye_right    = np.mean(lm_eye_right, axis=0)
        eye_avg      = (eye_left + eye_right) * 0.5
        eye_to_eye   = eye_right - eye_left
        mouth_avg    = (mouth_left_point + mount_right_point) * 0.5
        eye_to_mouth = mouth_avg - eye_avg

        # Choose oriented crop rectangle.
        x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
        x /= np.hypot(*x)
        x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
        x *= x_scale
        y = np.flipud(x) * [-y_scale, y_scale]
        c = eye_avg + eye_to_mouth * em_scale
        quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
        qsize = np.hypot(*x) * 2

        # Shrink.
        shrink = int(np.floor(qsize / output_size * 0.5))
        if shrink > 1:
            rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
            img = img.resize(rsize, Image.ANTIALIAS)
            quad /= shrink
            qsize /= shrink

        # Crop.
        border = max(int(np.rint(qsize * 0.1)), 3)
        crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
        if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
            img = img.crop(crop)
            quad -= crop[0:2]

        # Pad.
        pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
        if enable_padding and max(pad) > border - 4:
            pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
            img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
            h, w, _ = img.shape
            y, x, _ = np.ogrid[:h, :w, :1]
            mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
            blur = qsize * 0.02
            img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
            img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
            img = np.uint8(np.clip(np.rint(img), 0, 255))
            if alpha:
                mask = 1-np.clip(3.0 * mask, 0.0, 1.0)
                mask = np.uint8(np.clip(np.rint(mask*255), 0, 255))
                img = np.concatenate((img, mask), axis=2)
                img = Image.fromarray(img, 'RGBA')
            else:
                img = Image.fromarray(img, 'RGB')
            quad += pad[:2]

        # Transform.
        img = img.transform((transform_size, transform_size), Image.QUAD, (quad + 0.5).flatten(), Image.BILINEAR)
        if output_size < transform_size:
            img = img.resize((output_size, output_size), Image.ANTIALIAS)

        # Save aligned image.
        img.save(f'{Path(img_path).stem}_{i}.png', 'PNG')

In [20]:
align_image('/content/anime_face_landmark_detection/test.png')